In [1]:
import os,sys
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *
from tqdm import tqdm
from collections import Counter
import pandas as pd
import shutil
import pickle
import gc, math
from math import log, floor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import *
from scipy import sparse
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
max_words = 23 #dict size (tri-gram總數量)
max_length = 115 #第一轉折點 tri-gram num (最靠近outlier數字的斜率變化轉折點) 超過此數量的hkl拿掉
min_length = 30 #最小的轉折點(第三個轉折點) #Q1

In [3]:
root_dir = "./data/Hooklog/MikeGithub/"
fam_dirs = next(os.walk(root_dir))[1]
trainX = []
trainY = []
for fam in tqdm(fam_dirs):
    in_directory = root_dir + fam + '/'
    hl_list = next(os.walk(in_directory))[2] # get all filenames in the in_directory
    hl_list = [os.path.join(in_directory, f) for f in hl_list] # filepathname list
#     hl_list = list(filter(lambda f: f.endswith(".enc.npy"), hl_list)) # in case some non-hooklog file in the folder
    hl_list = list(filter(lambda f: f.endswith(".single.npy"), hl_list)) #load norm data
    for npy in hl_list:
        tri_enc = np.load(npy)
        if len(tri_enc)>max_length:
            tri_enc = tri_enc[:max_length]
        if len(tri_enc)<min_length:
            continue
        trainY.append(fam)
        trainX.append(tri_enc)
print(len(trainY),len(trainX))

100%|██████████| 13/13 [00:00<00:00, 121.04it/s]

486 486


In [4]:
fam_code = {}
for i,v in enumerate(list(set(trainY))):
    fam_code[v] = i
# fam_code = pickle.load(file=open(root_dir+"fam_dict_enc.pkl",'rb'))
train_Y = []
for fam in trainY:
    train_Y.append(fam_code[fam])
train_Y = np.array(train_Y)
pickle.dump(obj=fam_code,file=open(root_dir+"fam_dict_enc_single_Mike.pkl",'wb'))
train_Y

array([ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3

In [5]:
#encode to int (remember the encoding)
train_X = tf.keras.preprocessing.sequence.pad_sequences(trainX,maxlen=max_length,padding='post',value=0,dtype='int32')
print(train_X.shape, train_Y.shape)

(486, 115) (486,)


In [6]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

In [14]:
X_train, y_train, X_test, y_test = split_valid_set(train_X, train_Y, 0.9)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
assert len(set(y_test)) == len(set(y_train))
x = dict(Counter(y_test))
temp = {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}
assert temp[list(temp.keys())[0]] > 1
print(set(y_test))
# if assertion error, run more time for this cell

(437, 115) (437,) (49, 115) (49,)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


# Build Model

In [15]:
emb_dim = 128#validXr.shape[-1] 
max_length = X_test.shape[-1] # max sequence length
cat_num = y_test.shape[0]
do = 0.1
vocabulary_size = max_words+1
hidden_dim = 64
EPOCHS=2000
# traina = True #改
batch_size = 128 #改

In [24]:
#從0開始
# init_w = tensorflow.keras.initializers.RandomNormal(mean=0.9,stddev=0.06)
# init_b = tensorflow.keras.initializers.RandomNormal(mean=0.7,stddev=0.2)
#從1開始
init_w = tensorflow.keras.initializers.RandomNormal(mean=1,stddev=0.5)
init_b = tensorflow.keras.initializers.RandomNormal(mean=1.,stddev=0.5)
def onezero(x):
    portion = 1.0#0.6#0.6 #0.6~1
    z = tf.where(x>=1.0, x - x + 1.0, x)
    y = tf.where(z<=0.0, z - z + 0.0, portion*z)
    return y
train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(X_train.shape[0],reshuffle_each_iteration=True).batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(batch_size)



In [35]:
class base_model_1(Model):
    def __init__(self):
        super(base_model_1, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(vocabulary_size+1, emb_dim,input_length=max_length
                             ,trainable=True,name='embedding')
        self.rnn1 = GRU(emb_dim,return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.att = Attention(name='selfatt')
        self.bn1 = BatchNormalization(name='bn1')
        #self.fil = Dense(1,activation=onezero,name='filter_out')
        self.fil = TimeDistributed(Dense(1,activation=onezero,kernel_initializer=init_w,bias_initializer=init_b, name='filter_out'),name='TD2') #relu/linear/step function

    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        #x = self.att([x1,x1])
        x = self.rnn1(x1)
        x = self.bn1(x)
        x = self.att([x,x1,x1])
        y = self.fil(x)
        return x1,y

model1 = base_model_1()
# model1.load_weights(saveP1)
model1(X_train[:2])

(<tf.Tensor: shape=(2, 115, 128), dtype=float32, numpy=
 array([[[ 0.02923706,  0.01436955,  0.0255849 , ..., -0.01498837,
          -0.01974367,  0.02110518],
         [-0.03582343,  0.02793909, -0.03459384, ..., -0.04751979,
           0.0427042 ,  0.03811348],
         [-0.00757133,  0.00692255,  0.0070165 , ..., -0.04625129,
          -0.04621822,  0.01490918],
         ...,
         [-0.03582343,  0.02793909, -0.03459384, ..., -0.04751979,
           0.0427042 ,  0.03811348],
         [ 0.02552046, -0.00729968,  0.01688388, ...,  0.0001887 ,
          -0.01672114,  0.02061741],
         [ 0.02923706,  0.01436955,  0.0255849 , ..., -0.01498837,
          -0.01974367,  0.02110518]],
 
        [[-0.03582343,  0.02793909, -0.03459384, ..., -0.04751979,
           0.0427042 ,  0.03811348],
         [-0.03582343,  0.02793909, -0.03459384, ..., -0.04751979,
           0.0427042 ,  0.03811348],
         [-0.03582343,  0.02793909, -0.03459384, ..., -0.04751979,
           0.0427042 ,  0.03

In [44]:
class base_model_2(Model):
    def __init__(self):
        super(base_model_2, self).__init__()
        self.mul = Multiply()
        #self.rnn2 = Bidirectional(LSTM(int(hidden_dim),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = GRU(int(hidden_dim/4))
        self.bn2 = BatchNormalization(name='bn2')
        self.dn = Dense(int(hidden_dim/4),activation='selu',
                        kernel_initializer=tf.keras.initializers.lecun_normal(), name='dn')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')

    def call(self,x1,y1):
        x2 = self.mul([y1,x1])
        x = self.rnn3(x2) #x2 #y1=weight|binary
        #x = self.bn2(x2)
        #x = self.dn(x)
        y2 = self.out(x)
        return y2
    
model2 = base_model_2()

In [45]:
seq_num = 2 #連續幾個才叫做連續，要改modify_idx看有幾個

arr_len = max_length - seq_num + 1
seq_arr = []
for i in range(arr_len):
    ori_np = np.array([0]*max_length)
    modify_idx = [i,i+1] #要跟著seq_num改
    ori_np[modify_idx]=1
    seq_arr.append(ori_np)
seq_arr = np.array(seq_arr)
seq_mask = tf.Variable(seq_arr.T,dtype='float32')
seq_mask = tf.expand_dims(seq_mask,axis=0)
seq_mask

<tf.Tensor: shape=(1, 115, 114), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>

In [46]:
def loss_object1(predictions): #filter loss
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def one_percentage(predictions): #1 num
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def seq_loss(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    predictions = tf.squeeze(predictions,axis=-1)
    results = tf.matmul(predictions,seq_mask)
    results = tf.where(results==seq_num,1.0,0.0)
    loss_ = tf.reduce_mean(results)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def word_reproduce1(prediction_int):
    return prediction_int
def word_reproduce2(prediction_int):
    return prediction_int
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy() #clf loss

# optimizer1 = tf.keras.optimizers.SGD(learning_rate=0.0044, momentum=0.5, nesterov=True)
optimizer1 = tf.keras.optimizers.Nadam()#Adam(learning_rate=0.0003)

In [47]:
train_loss = tf.keras.metrics.Mean(name='train_loss') #total_loss
train_accloss = tf.keras.metrics.Mean(name='train_accloss')#loss_acc
train_filloss = tf.keras.metrics.Mean(name='train_filloss') #loss_filter
train_seqloss = tf.keras.metrics.Mean(name='train_seqloss') #loss_seq
train_ones = tf.keras.metrics.Mean(name='train_ones') #ones_num
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') #acc_rate

test_loss = tf.keras.metrics.Mean(name='test_loss') #total_loss
test_accloss = tf.keras.metrics.Mean(name='test_accloss')#loss_acc
test_filloss = tf.keras.metrics.Mean(name='test_filloss') #loss_filter
test_seqloss = tf.keras.metrics.Mean(name='test_seqloss')
test_ones = tf.keras.metrics.Mean(name='test_ones') #ones_num
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [48]:
alpha = 0.0#0.00000001#0.000002 #pahse1: -0.1 / 0.0 ; phase2: 0.01~0.05~0.1 有1-就是希望1越多，沒1-就是希望0越多1越少
beta = 1.0 #clf loss 越大越要求分好
gamma = 0.0 #seqloss 越大越要求連續
#-0.001 / 1.0 / 1.0

@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=False) as tape:
        emb, pred_imp = model1(x)
        #loss1 = alpha*loss_object1(pred_imp) #phase1
        #pred_imp2 = tf.math.round(pred_imp)
        #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
        pred_imp2 = tf.math.round(pred_imp)
        loss1 = loss_object1(pred_imp) #有1-就是希望1越多，沒1-就是希望0越多1越少 #pahse2:alpha*loss_object1(pred_imp) ; phase1: alpha*(1-loss_object1(pred_imp))
        pred_cat = model2(emb,pred_imp2) #weight: pred_imp; 0/1binary: pred_imp2 。#pahse1: pred_imp; phase2; pred_imp2
        loss2 = loss_object2(yc, pred_cat)
        loss3 = 1-seq_loss(pred_imp2)
        loss = alpha*loss1 + beta*loss2 + gamma*loss3
    trainable_variable = model1.trainable_variables
    trainable_variable.extend(model2.trainable_variables)
    gradients = tape.gradient(loss,trainable_variable)
    optimizer1.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_filloss(loss1)
    train_accloss(loss2)
    train_seqloss(loss3) #loss_seq
    train_accuracy(yc, pred_cat) #acc_rate
    ones = one_percentage(pred_imp2) #pred_imp2
    train_ones(ones) #ones_num
    
    
@tf.function
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #weight: pred_imp ; binary:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)

    
# word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
# word_idx_pd['ori_word'] = word_idx_pd.index 
# word_idx_pd = word_idx_pd.set_index(["ID"])
# print(word_idx_pd)

ep = -1
@tf.function(experimental_relax_shapes=True)
def test_step_inf(x,yc,epoch):
    #ori_text = getKeysByValues(list(x))
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    #if ep != epoch:
        #ep = epoch
        #x = tf.dtypes.cast(x,dtype=tf.float32)
    pred_imp3_ = tf.squeeze(pred_imp2)
    pred_imp3 = tf.dtypes.cast(pred_imp3_,dtype=tf.int32)
    used_x = tf.math.multiply(x,pred_imp3)
    filter_emb = tf.math.multiply(emb,pred_imp2)
        #word_reproduce1(x)
        #word_reproduce2(used_x)
        #used_text = getKeysByValues(list(used_x))
        #tf.print(used_x)
        #print(emb.numpy(),pred_imp, x,pred_imp3,used_x)
        #int_ori_text.append(x)
        #int_fil_text.append(used_x)
    
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #weight: pred_imp ; binary:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
    return x, used_x, pred_cat, emb, filter_emb

In [ ]:
EPOCHS = 2000
DateID = '2020020201'

saveP1 = './model/Hooklog/'+DateID+'/model1'
saveP2 = './model/Hooklog/'+DateID+'/model2'
train_loss_acc = []
train_loss_filter = []
train_loss_seq = []
train_weighted_loss = []
train_acc_rate = []
train_ones_num = []

test_loss_acc = []
test_loss_filter = []
test_loss_seq = []
test_weighted_loss = []
test_acc_rate = []
test_ones_num = []

int_ori_text =[]
int_fil_text = []
int_label = []
pred_label = []
ori_emb = []
fil_emb = []
epoch_num = []
fam_lab = []
pred_lab = []
# signature_dict = {'att':model1.att}

gc.collect()
best_clf = 0.0
for epoch in range(EPOCHS):
    epoch = tf.convert_to_tensor(epoch, dtype=tf.int64)
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        ori,fil,pred_cat,emb,filter_emb = test_step_inf(test_text, test_labels,epoch) #with restore words
        ori_emb.append(emb.numpy()) #一個epoch有很多個batches
        fil_emb.append(filter_emb.numpy()) #一個epoch有很多個batches
        epoch_num.append(epoch.numpy()) #一個epoch一個數字
        fam_lab.append(test_labels.numpy())#一個epoch有很多個batches
        pred_lab.append(pred_cat.numpy())#一個epoch有很多個batches
        #print(k2)
        #test_step(test_text, test_labels) #original
    int_ori_text.append(ori.numpy())
    int_fil_text.append(fil.numpy())
    int_label.append(test_labels.numpy()) #true label
    pred_label.append(pred_cat.numpy())

    
    template = 'Epoch {}, Total Loss: {}, Clf Loss: {}, Filter Loss: {}, Seq Loss: {}, Accuracy Rate: {:5.2f}%, Ones Portion: {}, \
            Test_Total_Loss: {}, Test_Clf_Loss: {}, Test_Filter_Loss: {}, TEST_Seq_Loss: {}, Test_Accuracy_Rate: {:5.2f}%, Test_Ones_Portion: {}'
    print(template.format(epoch+1,train_loss.result(),
                          train_accloss.result(),train_filloss.result(),train_seqloss.result(),
                        train_accuracy.result()*100,train_ones.result(),
                        test_loss.result(),
                        test_accloss.result(),test_filloss.result(),test_seqloss.result(),
                        test_accuracy.result()*100,test_ones.result(),
                        ))
    #int_ori_text.append(word_reproduce1.result().numpy())
    #int_fil_text.append(word_reproduce2.result().numpy())
    train_loss_acc.append( train_accloss.result().numpy())
    train_loss_filter.append( train_filloss.result().numpy())
    train_loss_seq.append( train_seqloss.result().numpy())
    train_weighted_loss.append( train_loss.result().numpy())
    train_acc_rate.append( train_accuracy.result().numpy())
    train_ones_num.append( train_ones.result().numpy())
    
    test_loss_acc.append( test_accloss.result().numpy())
    test_loss_filter.append( test_filloss.result().numpy())
    test_loss_seq.append( test_seqloss.result().numpy())
    test_weighted_loss.append( test_loss.result().numpy())
    test_acc_rate.append( test_accuracy.result().numpy())
    test_ones_num.append( test_ones.result().numpy())
    if best_clf<=test_accuracy.result()*100:
        #tf.saved_model.save(model1,saveP1+'_all')
        #model1.save(saveP1,save_format='h5')
        #model2.save(saveP2,save_format='h5')
        #tf.saved_model.save(model2,saveP2+'_all')
        model1.save_weights(saveP1,save_format='tf')
        model2.save_weights(saveP2,save_format='tf')
        best_clf = test_accuracy.result()*100
        print('===MODEL WEIGHTS SAVED===',saveP1,saveP2)
    # Reset the metrics for the next epoch
    #word_reproduce1.reset_states()
    #word_reproduce2.reset_states()
    train_loss.reset_states()
    train_accloss.reset_states()
    train_filloss.reset_states()
    train_seqloss.reset_states()
    train_ones.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accloss.reset_states()
    test_filloss.reset_states()
    test_seqloss.reset_states()
    test_ones.reset_states()
    test_accuracy.reset_states()
pred_lab2 = []
for pred in pred_lab:
    pred_lab2.append(list(np.argmax(pred,axis=-1)))

Epoch 1, Total Loss: 2.4784157276153564, Clf Loss: 2.4784157276153564, Filter Loss: 0.47154611349105835, Seq Loss: 0.902978777885437, Accuracy Rate: 10.53%, Ones Portion: 0.09704336524009705,             Test_Total_Loss: 2.476612091064453, Test_Clf_Loss: 2.476612091064453, Test_Filter_Loss: 0.4498422145843506, TEST_Seq_Loss: 0.9733444452285767, Test_Accuracy_Rate: 22.45%, Test_Ones_Portion: 0.02665555849671364
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hooklog/2020020201/model2
Epoch 2, Total Loss: 2.46565842628479, Clf Loss: 2.46565842628479, Filter Loss: 0.4659949541091919, Seq Loss: 0.9339184165000916, Accuracy Rate: 28.83%, Ones Portion: 0.06608159095048904,             Test_Total_Loss: 2.467897415161133, Test_Clf_Loss: 2.467897415161133, Test_Filter_Loss: 0.4490496814250946, TEST_Seq_Loss: 0.9733444452285767, Test_Accuracy_Rate: 22.45%, Test_Ones_Portion: 0.02665555849671364
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hooklog/202002

Epoch 19, Total Loss: 1.80862557888031, Clf Loss: 1.80862557888031, Filter Loss: 0.4927806258201599, Seq Loss: 0.8403773903846741, Accuracy Rate: 31.81%, Ones Portion: 0.16137820482254028,             Test_Total_Loss: 2.0380756855010986, Test_Clf_Loss: 2.0380756855010986, Test_Filter_Loss: 0.4666063189506531, TEST_Seq_Loss: 0.9011014699935913, Test_Accuracy_Rate: 18.37%, Test_Ones_Portion: 0.10034335404634476
Epoch 20, Total Loss: 1.7860957384109497, Clf Loss: 1.7860957384109497, Filter Loss: 0.49065178632736206, Seq Loss: 0.845177173614502, Accuracy Rate: 31.81%, Ones Portion: 0.15689939260482788,             Test_Total_Loss: 2.0250906944274902, Test_Clf_Loss: 2.0250906944274902, Test_Filter_Loss: 0.4641856253147125, TEST_Seq_Loss: 0.9085224866867065, Test_Accuracy_Rate: 18.37%, Test_Ones_Portion: 0.0927356481552124
Epoch 21, Total Loss: 1.7795566320419312, Clf Loss: 1.7795566320419312, Filter Loss: 0.4897409677505493, Seq Loss: 0.8549060821533203, Accuracy Rate: 31.81%, Ones Portion:

Epoch 39, Total Loss: 1.6691820621490479, Clf Loss: 1.6691820621490479, Filter Loss: 0.48481857776641846, Seq Loss: 0.8595858216285706, Accuracy Rate: 35.01%, Ones Portion: 0.14262783527374268,             Test_Total_Loss: 1.8704899549484253, Test_Clf_Loss: 1.8704899549484253, Test_Filter_Loss: 0.45899781584739685, TEST_Seq_Loss: 0.8947235345840454, Test_Accuracy_Rate: 20.41%, Test_Ones_Portion: 0.10673817247152328
Epoch 40, Total Loss: 1.6563122272491455, Clf Loss: 1.6563122272491455, Filter Loss: 0.4860561490058899, Seq Loss: 0.8682064414024353, Accuracy Rate: 35.24%, Ones Portion: 0.13420367240905762,             Test_Total_Loss: 1.8650002479553223, Test_Clf_Loss: 1.8650002479553223, Test_Filter_Loss: 0.4600119888782501, TEST_Seq_Loss: 0.8865278363227844, Test_Accuracy_Rate: 20.41%, Test_Ones_Portion: 0.11464864015579224
Epoch 41, Total Loss: 1.6704038381576538, Clf Loss: 1.6704038381576538, Filter Loss: 0.4845065176486969, Seq Loss: 0.8620331287384033, Accuracy Rate: 35.93%, Ones P

Epoch 59, Total Loss: 1.6119942665100098, Clf Loss: 1.6119942665100098, Filter Loss: 0.4929178059101105, Seq Loss: 0.9002419710159302, Accuracy Rate: 36.61%, Ones Portion: 0.10131760686635971,             Test_Total_Loss: 1.8229217529296875, Test_Clf_Loss: 1.8229217529296875, Test_Filter_Loss: 0.46194547414779663, TEST_Seq_Loss: 0.9224058389663696, Test_Accuracy_Rate: 26.53%, Test_Ones_Portion: 0.07921702414751053
Epoch 60, Total Loss: 1.6112785339355469, Clf Loss: 1.6112785339355469, Filter Loss: 0.49708086252212524, Seq Loss: 0.89743572473526, Accuracy Rate: 38.67%, Ones Portion: 0.10417522490024567,             Test_Total_Loss: 1.819454550743103, Test_Clf_Loss: 1.819454550743103, Test_Filter_Loss: 0.46417874097824097, TEST_Seq_Loss: 0.9219070672988892, Test_Accuracy_Rate: 28.57%, Test_Ones_Portion: 0.07961712032556534
Epoch 61, Total Loss: 1.6225031614303589, Clf Loss: 1.6225031614303589, Filter Loss: 0.4938930869102478, Seq Loss: 0.9034214615821838, Accuracy Rate: 37.53%, Ones Port

Epoch 76, Total Loss: 1.5172723531723022, Clf Loss: 1.5172723531723022, Filter Loss: 0.5291379690170288, Seq Loss: 0.8697416186332703, Accuracy Rate: 53.55%, Ones Portion: 0.13286402821540833,             Test_Total_Loss: 1.684061050415039, Test_Clf_Loss: 1.684061050415039, Test_Filter_Loss: 0.49454954266548157, TEST_Seq_Loss: 0.9030495882034302, Test_Accuracy_Rate: 42.86%, Test_Ones_Portion: 0.09799621999263763
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hooklog/2020020201/model2
Epoch 77, Total Loss: 1.4772109985351562, Clf Loss: 1.4772109985351562, Filter Loss: 0.5331673622131348, Seq Loss: 0.8581959009170532, Accuracy Rate: 53.55%, Ones Portion: 0.14449435472488403,             Test_Total_Loss: 1.749464988708496, Test_Clf_Loss: 1.749464988708496, Test_Filter_Loss: 0.4917915463447571, TEST_Seq_Loss: 0.9255498051643372, Test_Accuracy_Rate: 38.78%, Test_Ones_Portion: 0.07595477253198624
Epoch 78, Total Loss: 1.5824894905090332, Clf Loss: 1.5824894905090332, Fil

...

Epoch 240, Total Loss: 1.1529204845428467, Clf Loss: 1.1529204845428467, Filter Loss: 0.410819411277771, Seq Loss: 0.8059532642364502, Accuracy Rate: 55.15%, Ones Portion: 0.1945171356201172,             Test_Total_Loss: 1.0639334917068481, Test_Clf_Loss: 1.0639334917068481, Test_Filter_Loss: 0.43162801861763, TEST_Seq_Loss: 0.7975372076034546, Test_Accuracy_Rate: 61.22%, Test_Ones_Portion: 0.2026996910572052
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hooklog/2020020201/model2
Epoch 241, Total Loss: 1.1313436031341553, Clf Loss: 1.1313436031341553, Filter Loss: 0.4108470380306244, Seq Loss: 0.8047595024108887, Accuracy Rate: 55.15%, Ones Portion: 0.19566966593265533,             Test_Total_Loss: 1.063632607460022, Test_Clf_Loss: 1.063632607460022, Test_Filter_Loss: 0.42959296703338623, TEST_Seq_Loss: 0.7976975440979004, Test_Accuracy_Rate: 61.22%, Test_Ones_Portion: 0.20253993570804596
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hooklog/

Epoch 257, Total Loss: 1.1155277490615845, Clf Loss: 1.1155277490615845, Filter Loss: 0.40032708644866943, Seq Loss: 0.8025305271148682, Accuracy Rate: 54.69%, Ones Portion: 0.19799044728279114,             Test_Total_Loss: 1.0617928504943848, Test_Clf_Loss: 1.0617928504943848, Test_Filter_Loss: 0.4187876880168915, TEST_Seq_Loss: 0.7975372076034546, Test_Accuracy_Rate: 61.22%, Test_Ones_Portion: 0.2026996910572052
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hooklog/2020020201/model2
Epoch 258, Total Loss: 1.1254633665084839, Clf Loss: 1.1254633665084839, Filter Loss: 0.3983210623264313, Seq Loss: 0.8057451248168945, Accuracy Rate: 54.69%, Ones Portion: 0.1947031021118164,             Test_Total_Loss: 1.0640993118286133, Test_Clf_Loss: 1.0640993118286133, Test_Filter_Loss: 0.41857144236564636, TEST_Seq_Loss: 0.7976975440979004, Test_Accuracy_Rate: 61.22%, Test_Ones_Portion: 0.20253993570804596
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Ho

Epoch 274, Total Loss: 1.1216764450073242, Clf Loss: 1.1216764450073242, Filter Loss: 0.3921506404876709, Seq Loss: 0.80706787109375, Accuracy Rate: 54.69%, Ones Portion: 0.19344556331634521,             Test_Total_Loss: 1.0536644458770752, Test_Clf_Loss: 1.0536644458770752, Test_Filter_Loss: 0.4140988886356354, TEST_Seq_Loss: 0.7976975440979004, Test_Accuracy_Rate: 61.22%, Test_Ones_Portion: 0.20253993570804596
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hooklog/2020020201/model2
Epoch 275, Total Loss: 1.1164991855621338, Clf Loss: 1.1164991855621338, Filter Loss: 0.3924320340156555, Seq Loss: 0.8059406280517578, Accuracy Rate: 54.69%, Ones Portion: 0.19456692039966583,             Test_Total_Loss: 1.0537348985671997, Test_Clf_Loss: 1.0537348985671997, Test_Filter_Loss: 0.4146500825881958, TEST_Seq_Loss: 0.7976975440979004, Test_Accuracy_Rate: 61.22%, Test_Ones_Portion: 0.20253993570804596
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hook

Epoch 291, Total Loss: 1.1221234798431396, Clf Loss: 1.1221234798431396, Filter Loss: 0.3967090845108032, Seq Loss: 0.8015231490135193, Accuracy Rate: 54.69%, Ones Portion: 0.19901275634765625,             Test_Total_Loss: 1.056033968925476, Test_Clf_Loss: 1.056033968925476, Test_Filter_Loss: 0.40847247838974, TEST_Seq_Loss: 0.7976975440979004, Test_Accuracy_Rate: 61.22%, Test_Ones_Portion: 0.20253993570804596
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hooklog/2020020201/model2
Epoch 292, Total Loss: 1.093029499053955, Clf Loss: 1.093029499053955, Filter Loss: 0.394153892993927, Seq Loss: 0.8044981956481934, Accuracy Rate: 54.69%, Ones Portion: 0.1961304396390915,             Test_Total_Loss: 1.0542235374450684, Test_Clf_Loss: 1.0542235374450684, Test_Filter_Loss: 0.40852001309394836, TEST_Seq_Loss: 0.7976975440979004, Test_Accuracy_Rate: 61.22%, Test_Ones_Portion: 0.20253993570804596
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hooklog/2

Epoch 308, Total Loss: 1.0926355123519897, Clf Loss: 1.0926355123519897, Filter Loss: 0.38909462094306946, Seq Loss: 0.7943587303161621, Accuracy Rate: 55.15%, Ones Portion: 0.20632322132587433,             Test_Total_Loss: 1.0505475997924805, Test_Clf_Loss: 1.0505475997924805, Test_Filter_Loss: 0.40517500042915344, TEST_Seq_Loss: 0.7976975440979004, Test_Accuracy_Rate: 61.22%, Test_Ones_Portion: 0.20253993570804596
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model/Hooklog/2020020201/model2
Epoch 309, Total Loss: 1.1225526332855225, Clf Loss: 1.1225526332855225, Filter Loss: 0.37918591499328613, Seq Loss: 0.8150262832641602, Accuracy Rate: 55.15%, Ones Portion: 0.18547220528125763,             Test_Total_Loss: 1.0484510660171509, Test_Clf_Loss: 1.0484510660171509, Test_Filter_Loss: 0.4053390324115753, TEST_Seq_Loss: 0.7976975440979004, Test_Accuracy_Rate: 61.22%, Test_Ones_Portion: 0.20253993570804596
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020201/model1 ./model

In [9]:
"""
class test_model(Model):
    def __init__(self):
        super(test_model, self).__init__()
        self.emb = Embedding(vocabulary_size+1, emb_dim,input_length=max_length
                             ,trainable=True,name='embedding')
        self.rnn = LSTM(hidden_dim,return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn1 = BatchNormalization(name='bn1')
        self.bn2 = BatchNormalization(name='bn2')
        self.att = Attention(name='selfatt')
        self.rnn2 = GRU(int(hidden_dim/2),name='common_extract2'
                      ,trainable=True)
        self.dn = Dense(int(hidden_dim/4),activation='selu',
                        kernel_initializer=tf.keras.initializers.lecun_normal(), name='dn')
        
        self.cnn = Conv1D(int(hidden_dim/2),strides=2,kernel_size=4)
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')
        #self.out = Dense(max(train_Y)+1,activation='softmax',name='clf')
        self.flat = Flatten()
    def call(self,x):
        x1 = self.emb(x)
        #x = self.cnn(x)
        x = self.att([x1,x1,x1])
        #x = self.rnn(x)
        #x = self.bn1(x)
        #x = self.att([x,x1,x1])
        #x = self.att([x,x,x])
        x = self.rnn2(x)
        x = self.bn2(x)
        #x = self.flat(x)
        x = self.dn(x)
        y2 = self.out(x)
        return y2
model = test_model()
# batch_size = 32
try:
    train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(X_train.shape[0],reshuffle_each_iteration=True).batch(batch_size)
    valid_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(batch_size)
except NameError:
    train_ds = tf.data.Dataset.from_tensor_slices((train_X,train_Y)).shuffle(train_X.shape[0]).batch(batch_size)
    valid_ds = tf.data.Dataset.from_tensor_slices((train_X,train_Y)).batch(batch_size)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Nadam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') 
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(x,yc):
    with tf.GradientTape() as tape:
        pred_cat = model(x) #weight: pred_imp; 0/1binary: pred_imp2 。#pahse1: pred_imp; phase2; pred_imp2
        loss = loss_object(yc, pred_cat)
    trainable_variable = model.trainable_variables
    gradients = tape.gradient(loss,trainable_variable)
    optimizer.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_accuracy(yc, pred_cat) #acc_rate

@tf.function
def test_step(x,yc):
    pred_cat = model(x) #weight: pred_imp ; binary:pred_imp2
    t_loss = loss_object2(yc, pred_cat)
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)


for epoch in range(EPOCHS):
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        test_step(test_text, test_labels) #with restore words
    
    template = 'Epoch {}, Total Loss: {}, Accuracy Rate: {:5.2f}%,\
            Test_Total_Loss: {}, Test_Accuracy_Rate: {:5.2f}%'
    print(template.format(epoch+1,train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100
                        ))

    train_loss.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accuracy.reset_states()
"""    

Epoch 1, Total Loss: 2.4634788036346436, Accuracy Rate: 16.02%,            Test_Total_Loss: 2.4256539344787598, Test_Accuracy_Rate: 34.69%
Epoch 2, Total Loss: 2.423973560333252, Accuracy Rate: 27.46%,            Test_Total_Loss: 2.3692984580993652, Test_Accuracy_Rate: 44.90%
Epoch 3, Total Loss: 2.374782085418701, Accuracy Rate: 31.58%,            Test_Total_Loss: 2.2945475578308105, Test_Accuracy_Rate: 44.90%
Epoch 4, Total Loss: 2.3063619136810303, Accuracy Rate: 33.87%,            Test_Total_Loss: 2.194746255874634, Test_Accuracy_Rate: 44.90%
Epoch 5, Total Loss: 2.2172152996063232, Accuracy Rate: 33.87%,            Test_Total_Loss: 2.0625734329223633, Test_Accuracy_Rate: 44.90%
Epoch 6, Total Loss: 2.100715160369873, Accuracy Rate: 33.87%,            Test_Total_Loss: 1.9044348001480103, Test_Accuracy_Rate: 44.90%
Epoch 7, Total Loss: 1.988112211227417, Accuracy Rate: 33.87%,            Test_Total_Loss: 1.773451805114746, Test_Accuracy_Rate: 44.90%
Epoch 8, Total Loss: 1.8965494632

Epoch 60, Total Loss: 0.315942645072937, Accuracy Rate: 92.68%,            Test_Total_Loss: 0.3443942666053772, Test_Accuracy_Rate: 89.80%
Epoch 61, Total Loss: 0.3571322560310364, Accuracy Rate: 92.91%,            Test_Total_Loss: 0.3495045006275177, Test_Accuracy_Rate: 91.84%
Epoch 62, Total Loss: 0.2994227409362793, Accuracy Rate: 94.28%,            Test_Total_Loss: 0.32165268063545227, Test_Accuracy_Rate: 89.80%
Epoch 63, Total Loss: 0.3033389449119568, Accuracy Rate: 94.05%,            Test_Total_Loss: 0.33033356070518494, Test_Accuracy_Rate: 91.84%
Epoch 64, Total Loss: 0.2724860906600952, Accuracy Rate: 95.42%,            Test_Total_Loss: 0.3141513168811798, Test_Accuracy_Rate: 93.88%
Epoch 65, Total Loss: 0.25926631689071655, Accuracy Rate: 95.88%,            Test_Total_Loss: 0.305127888917923, Test_Accuracy_Rate: 93.88%
Epoch 66, Total Loss: 0.2555406987667084, Accuracy Rate: 96.11%,            Test_Total_Loss: 0.29319852590560913, Test_Accuracy_Rate: 93.88%
Epoch 67, Total Lo

Epoch 118, Total Loss: 0.11680293083190918, Accuracy Rate: 97.94%,            Test_Total_Loss: 0.16955511271953583, Test_Accuracy_Rate: 97.96%
Epoch 119, Total Loss: 0.10913911461830139, Accuracy Rate: 97.25%,            Test_Total_Loss: 0.1685483455657959, Test_Accuracy_Rate: 97.96%
Epoch 120, Total Loss: 0.11287971585988998, Accuracy Rate: 97.25%,            Test_Total_Loss: 0.16943176090717316, Test_Accuracy_Rate: 97.96%
Epoch 121, Total Loss: 0.12860016524791718, Accuracy Rate: 97.48%,            Test_Total_Loss: 0.17253278195858002, Test_Accuracy_Rate: 97.96%
Epoch 122, Total Loss: 0.11405998468399048, Accuracy Rate: 97.94%,            Test_Total_Loss: 0.16621461510658264, Test_Accuracy_Rate: 97.96%
Epoch 123, Total Loss: 0.10440023243427277, Accuracy Rate: 97.25%,            Test_Total_Loss: 0.16237540543079376, Test_Accuracy_Rate: 97.96%
Epoch 124, Total Loss: 0.1048857644200325, Accuracy Rate: 97.71%,            Test_Total_Loss: 0.1621246486902237, Test_Accuracy_Rate: 97.96%
Ep

Epoch 176, Total Loss: 0.06293867528438568, Accuracy Rate: 98.40%,            Test_Total_Loss: 0.17127573490142822, Test_Accuracy_Rate: 97.96%
Epoch 177, Total Loss: 0.06050989776849747, Accuracy Rate: 98.17%,            Test_Total_Loss: 0.17216740548610687, Test_Accuracy_Rate: 97.96%
Epoch 178, Total Loss: 0.06251807510852814, Accuracy Rate: 97.94%,            Test_Total_Loss: 0.17479440569877625, Test_Accuracy_Rate: 97.96%
Epoch 179, Total Loss: 0.06268482655286789, Accuracy Rate: 98.40%,            Test_Total_Loss: 0.17371796071529388, Test_Accuracy_Rate: 97.96%
Epoch 180, Total Loss: 0.0635482594370842, Accuracy Rate: 98.40%,            Test_Total_Loss: 0.16703957319259644, Test_Accuracy_Rate: 97.96%
Epoch 181, Total Loss: 0.056693386286497116, Accuracy Rate: 98.40%,            Test_Total_Loss: 0.1671418994665146, Test_Accuracy_Rate: 97.96%
Epoch 182, Total Loss: 0.05639089643955231, Accuracy Rate: 98.40%,            Test_Total_Loss: 0.16982212662696838, Test_Accuracy_Rate: 97.96%


Epoch 234, Total Loss: 0.04140513762831688, Accuracy Rate: 98.63%,            Test_Total_Loss: 0.28578561544418335, Test_Accuracy_Rate: 95.92%
Epoch 235, Total Loss: 0.04615527391433716, Accuracy Rate: 98.86%,            Test_Total_Loss: 0.28138813376426697, Test_Accuracy_Rate: 95.92%
Epoch 236, Total Loss: 0.0491412952542305, Accuracy Rate: 98.63%,            Test_Total_Loss: 0.2858261466026306, Test_Accuracy_Rate: 95.92%
Epoch 237, Total Loss: 0.04032721742987633, Accuracy Rate: 98.63%,            Test_Total_Loss: 0.2884337604045868, Test_Accuracy_Rate: 95.92%
Epoch 238, Total Loss: 0.04012273624539375, Accuracy Rate: 98.86%,            Test_Total_Loss: 0.290762335062027, Test_Accuracy_Rate: 95.92%
Epoch 239, Total Loss: 0.04377969354391098, Accuracy Rate: 98.86%,            Test_Total_Loss: 0.2904013693332672, Test_Accuracy_Rate: 95.92%
Epoch 240, Total Loss: 0.043402791023254395, Accuracy Rate: 98.63%,            Test_Total_Loss: 0.29120445251464844, Test_Accuracy_Rate: 95.92%
Epoc

...

Epoch 1736, Total Loss: 0.023253625258803368, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5252882242202759, Test_Accuracy_Rate: 95.92%
Epoch 1737, Total Loss: 0.024926483631134033, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5250384211540222, Test_Accuracy_Rate: 95.92%
Epoch 1738, Total Loss: 0.023281151428818703, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5246756076812744, Test_Accuracy_Rate: 95.92%
Epoch 1739, Total Loss: 0.02721536159515381, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5247055888175964, Test_Accuracy_Rate: 95.92%
Epoch 1740, Total Loss: 0.02350069209933281, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5251055359840393, Test_Accuracy_Rate: 95.92%
Epoch 1741, Total Loss: 0.025487594306468964, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5254871845245361, Test_Accuracy_Rate: 95.92%
Epoch 1742, Total Loss: 0.023361526429653168, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5255414843559265, Test_Accuracy_Rate: 9

Epoch 1794, Total Loss: 0.023704832419753075, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5313947796821594, Test_Accuracy_Rate: 95.92%
Epoch 1795, Total Loss: 0.023497240617871284, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5322796106338501, Test_Accuracy_Rate: 95.92%
Epoch 1796, Total Loss: 0.02481076307594776, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5319979786872864, Test_Accuracy_Rate: 95.92%
Epoch 1797, Total Loss: 0.02338649332523346, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5318813920021057, Test_Accuracy_Rate: 95.92%
Epoch 1798, Total Loss: 0.023532724007964134, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5320343971252441, Test_Accuracy_Rate: 95.92%
Epoch 1799, Total Loss: 0.023208368569612503, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5324548482894897, Test_Accuracy_Rate: 95.92%
Epoch 1800, Total Loss: 0.023271147161722183, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5328341722488403, Test_Accuracy_Rate: 9

Epoch 1852, Total Loss: 0.027349665760993958, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5354619026184082, Test_Accuracy_Rate: 95.92%
Epoch 1853, Total Loss: 0.023584600538015366, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5360878705978394, Test_Accuracy_Rate: 95.92%
Epoch 1854, Total Loss: 0.02647409401834011, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5366904139518738, Test_Accuracy_Rate: 95.92%
Epoch 1855, Total Loss: 0.024028221145272255, Accuracy Rate: 98.86%,            Test_Total_Loss: 0.5370890498161316, Test_Accuracy_Rate: 95.92%
Epoch 1856, Total Loss: 0.023742586374282837, Accuracy Rate: 99.08%,            Test_Total_Loss: 0.5373416543006897, Test_Accuracy_Rate: 95.92%
Epoch 1857, Total Loss: 0.025334985926747322, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5373088717460632, Test_Accuracy_Rate: 95.92%
Epoch 1858, Total Loss: 0.023411661386489868, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5372507572174072, Test_Accuracy_Rate: 

Epoch 1910, Total Loss: 0.03618849813938141, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5421084761619568, Test_Accuracy_Rate: 95.92%
Epoch 1911, Total Loss: 0.02365032583475113, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5433001518249512, Test_Accuracy_Rate: 95.92%
Epoch 1912, Total Loss: 0.03381954878568649, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5422941446304321, Test_Accuracy_Rate: 95.92%
Epoch 1913, Total Loss: 0.032600730657577515, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5397481322288513, Test_Accuracy_Rate: 95.92%
Epoch 1914, Total Loss: 0.02385149896144867, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5385597348213196, Test_Accuracy_Rate: 95.92%
Epoch 1915, Total Loss: 0.024404563009738922, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5387056469917297, Test_Accuracy_Rate: 95.92%
Epoch 1916, Total Loss: 0.0375627763569355, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5417248606681824, Test_Accuracy_Rate: 95.92

Epoch 1968, Total Loss: 0.5909732580184937, Accuracy Rate: 91.30%,            Test_Total_Loss: 0.3000276982784271, Test_Accuracy_Rate: 97.96%
Epoch 1969, Total Loss: 0.22344714403152466, Accuracy Rate: 97.03%,            Test_Total_Loss: 0.1972883641719818, Test_Accuracy_Rate: 97.96%
Epoch 1970, Total Loss: 0.15495839715003967, Accuracy Rate: 97.25%,            Test_Total_Loss: 0.1568981558084488, Test_Accuracy_Rate: 97.96%
Epoch 1971, Total Loss: 0.12124611437320709, Accuracy Rate: 97.48%,            Test_Total_Loss: 0.1694805920124054, Test_Accuracy_Rate: 97.96%
Epoch 1972, Total Loss: 0.14472034573554993, Accuracy Rate: 97.03%,            Test_Total_Loss: 0.18261964619159698, Test_Accuracy_Rate: 97.96%
Epoch 1973, Total Loss: 0.08271212130784988, Accuracy Rate: 97.48%,            Test_Total_Loss: 0.1943798065185547, Test_Accuracy_Rate: 97.96%
Epoch 1974, Total Loss: 0.06741930544376373, Accuracy Rate: 97.71%,            Test_Total_Loss: 0.20453014969825745, Test_Accuracy_Rate: 97.96

## Emb test
* cosine similarity => sklearn shape: (samplesN, features)

ori_emb
fil_emb
epoch_num
fam_lab
pred_lab2

* euclidean distance for intra-dist
* cosine similarity for inter-dist

In [215]:
recent_ep = -1
col = ['Epoch#','Original_Intra_Similarity_Avg','Filter_Intra_Similarity_Avg',
       'Original_Outra_Similarity','Filter_Outra_Similarity']
sim_df = pd.DataFrame(columns=col)
for ep, o_e, f_e, p_l, t_l in tqdm(zip(epoch_num,ori_emb,fil_emb,pred_lab2,fam_lab)): #take out each epoch
    if ep != recent_ep:
        whole_dict_ori = {}
        whole_dict_fil = {}
#         print('EP:',ep)
        recent_ep = ep
        count =  0
    for oe,fe,pl,tl in zip(o_e, f_e, p_l, t_l): #take out each process/sample in the epoch
        count+=1
        try: #依照每個epoch的true label family分
            temp1 = whole_dict_ori[tl] #put by true label
            temp1.append(oe)
            temp2 = whole_dict_fil[tl]
            temp2.append(fe)
            whole_dict_ori[tl] = temp1 #original embedding
            whole_dict_fil[tl] = temp2 #filter embedding
        except KeyError:
            temp1 = [oe]
            temp2 = [fe]
            whole_dict_ori[tl] = temp1 
            whole_dict_fil[tl] = temp2
        assert len(whole_dict_ori[tl]) == len(whole_dict_fil[tl]) > 0
        assert len(whole_dict_ori) == len(whole_dict_fil) > 0
    whole_dict_ori = {k: v for k, v in sorted(whole_dict_ori.items(), key=lambda item: item[0])}
    whole_dict_fil = {k: v for k, v in sorted(whole_dict_fil.items(), key=lambda item: item[0])}
    in_ori_score_li = []
    in_fil_score_li = []
    out_ori_emb_li = []
    out_fil_emb_li = []
    for k,v in whole_dict_ori.items(): #群內
        if len(v)<2: #某個家族只有一筆test sample
            score = 'NaN'
            continue
        emb = np.array(v)
#         emb2 = np.mean(emb,axis=1)
#         emb2 = np.mean(emb2,axis=0)
        emb = emb.reshape(len(v),-1)
        emb2 = np.mean(emb,axis=0)
        score = cosine_similarity(emb)
        score = np.mean(score)
        score = score - 1/len(v) #+ 1e-6*len(emb)
#         if score <= 1e-5:
#             score = 'NaN'
#             continue
        in_ori_score_li.append(score) #每個家族群內的相似度分數
        out_ori_emb_li.append(emb2)
    for k,v in whole_dict_fil.items(): #FIlter Embedding
        if len(v)<2: #某個家族只有一筆test sample
            score = 'NaN'
            continue
        emb = np.array(v)
        emb = emb.reshape(len(v),-1)
        emb2 = np.mean(emb,axis=0)
        score = cosine_similarity(emb)
        score = np.mean(score)
        score = score - 1/len(v) #+ 1e-6
        if score <= 1e-6:
            score = 'NaN'
            continue
        in_fil_score_li.append(score) #每個家族群內的相似度分數
        out_fil_emb_li.append(emb2)
    out_ori_score = cosine_similarity(np.array(out_ori_emb_li))
    out_ori_score = np.mean(out_ori_score)
    out_ori_score = out_ori_score - 1/len(out_ori_emb_li)
    out_fil_score = cosine_similarity(np.array(out_fil_emb_li))
    out_fil_score = np.mean(out_fil_score)
    out_fil_score = out_fil_score - 1/len(out_fil_emb_li)
    temp = pd.Series([str(int(ep)+1),np.mean(in_ori_score_li),np.mean(in_fil_score_li),out_ori_score,out_fil_score],col)
    sim_df = sim_df.append(temp,ignore_index=True)
sim_df

2000it [00:22, 90.71it/s]


,Epoch#,Original_Intra_Similarity_Avg,Filter_Intra_Similarity_Avg,Original_Outra_Similarity,Filter_Outra_Similarity
0,0,0.548994,0.548994,0.320756,3.207557e-01
1,1,0.549687,0.549687,0.317695,3.176952e-01
2,2,0.550923,0.550923,0.311467,3.114671e-01
3,3,0.552796,0.552796,0.302986,3.029862e-01
4,4,0.555114,0.555114,0.294371,2.943712e-01
...,...,...,...,...,...
1995,1995,0.554967,0.857475,0.333457,9.536743e-07
1996,1996,0.554964,0.857462,0.333433,-5.960464e-07
1997,1997,0.554970,0.857480,0.333460,2.384186e-07
1998,1998,0.554978,0.857493,0.333498,7.152557e-07


      ori1/ori2      fil1 /  fil2
mean:     1.0/0.9757028          1.0/0.027777778

reshape:0.96861625/0.6387043       0.96861625/0.027777784
* reshape is bettere


In [211]:
'''kk = np.array(whole_dict_fil[2])
kk = kk.reshape(len(whole_dict_fil[2]),-1)
kk = np.mean(kk,axis=0)
# kk = np.mean(kk,axis=1)
# kk = sparse.csr_matrix(kk)
# temp = cosine_similarity(kk)
# temp = cosine_similarity(kk,dense_output=False)
np.mean(temp)'''

0.96861625

# Store Process

In [216]:
save_dir = './results/Hooklog/'+DateID+'/'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir, exist_ok=True)
saveR = save_dir + 'losses_metrics.xlsx'
data = {'train total loss':train_weighted_loss, 'train acc loss':train_loss_acc,
        'train filter loss':train_loss_filter,'train seq loss':train_loss_seq,
        'train acc rate':train_acc_rate, 'train ones num':train_ones_num,
        'test total loss':test_weighted_loss, 'test acc loss':test_loss_acc,
        'test filter loss':test_loss_filter, 'test seq loss': test_loss_seq,
        'test acc rate':test_acc_rate, 'test ones num':test_ones_num
       }
df = pd.DataFrame(data)
df_c = df.merge(sim_df, left_index=True,right_index=True)
col = df_c.columns.tolist()
n_col = [col[12]] + col[0:12]+ col[13:]
df = df_c[n_col]
df.to_excel(saveR)
print(saveR)
df

./results/Hooklog/2020012101/losses_metrics.xlsx


,train total loss,train acc loss,train filter loss,train seq loss,train acc rate,train ones num,test total loss,test acc loss,test filter loss,test seq loss,test acc rate,test ones num
0,2.482105,2.482105,0.598262,0.000000,0.057208,1.000000,2.444333,2.444333,0.597253,0.000000,0.265306,1.000000
1,2.440567,2.440567,0.597006,0.000000,0.196796,1.000000,2.388728,2.388728,0.597342,0.000000,0.346939,1.000000
2,2.392396,2.392396,0.596791,0.000000,0.265446,1.000000,2.318130,2.318130,0.594105,0.000000,0.408163,1.000000
3,2.322960,2.322960,0.592026,0.000000,0.272311,1.000000,2.223286,2.223286,0.590216,0.000000,0.428571,1.000000
4,2.238850,2.238850,0.584462,0.000000,0.313501,1.000000,2.099025,2.099025,0.584394,0.000000,0.428571,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.996609,1.996609,0.095893,0.978485,0.274600,0.021521,1.845307,1.845307,0.132257,0.961175,0.346939,0.038942
1996,1.968696,1.968696,0.100493,0.977533,0.274600,0.022472,1.846353,1.846353,0.132259,0.961175,0.346939,0.038942
1997,1.948684,1.948684,0.107366,0.968717,0.274600,0.031288,1.847010,1.847010,0.132204,0.961175,0.346939,0.038942
1998,1.976816,1.976816,0.102491,0.978300,0.274600,0.021705,1.846184,1.846184,0.132106,0.961175,0.346939,0.038942


In [223]:
# col = df_c.columns.tolist()
# n_col = [col[12]] + col[0:12]+ col[13:]
# df_c[n_col]

,Epoch#,train total loss,train acc loss,train filter loss,train seq loss,train acc rate,train ones num,test total loss,test acc loss,test filter loss,test seq loss,test acc rate,test ones num,Original_Intra_Similarity_Avg,Filter_Intra_Similarity_Avg,Original_Outra_Similarity,Filter_Outra_Similarity
0,0,2.482105,2.482105,0.598262,0.000000,0.057208,1.000000,2.444333,2.444333,0.597253,0.000000,0.265306,1.000000,0.548994,0.548994,0.320756,3.207557e-01
1,1,2.440567,2.440567,0.597006,0.000000,0.196796,1.000000,2.388728,2.388728,0.597342,0.000000,0.346939,1.000000,0.549687,0.549687,0.317695,3.176952e-01
2,2,2.392396,2.392396,0.596791,0.000000,0.265446,1.000000,2.318130,2.318130,0.594105,0.000000,0.408163,1.000000,0.550923,0.550923,0.311467,3.114671e-01
3,3,2.322960,2.322960,0.592026,0.000000,0.272311,1.000000,2.223286,2.223286,0.590216,0.000000,0.428571,1.000000,0.552796,0.552796,0.302986,3.029862e-01
4,4,2.238850,2.238850,0.584462,0.000000,0.313501,1.000000,2.099025,2.099025,0.584394,0.000000,0.428571,1.000000,0.555114,0.555114,0.294371,2.943712e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,1.996609,1.996609,0.095893,0.978485,0.274600,0.021521,1.845307,1.845307,0.132257,0.961175,0.346939,0.038942,0.554967,0.857475,0.333457,9.536743e-07
1996,1996,1.968696,1.968696,0.100493,0.977533,0.274600,0.022472,1.846353,1.846353,0.132259,0.961175,0.346939,0.038942,0.554964,0.857462,0.333433,-5.960464e-07
1997,1997,1.948684,1.948684,0.107366,0.968717,0.274600,0.031288,1.847010,1.847010,0.132204,0.961175,0.346939,0.038942,0.554970,0.857480,0.333460,2.384186e-07
1998,1998,1.976816,1.976816,0.102491,0.978300,0.274600,0.021705,1.846184,1.846184,0.132106,0.961175,0.346939,0.038942,0.554978,0.857493,0.333498,7.152557e-07


In [83]:
labelID_dict = pickle.load(open("./data/Hooklog/MikeGithub/fam_dict_enc_single_Mike.pkl","rb"))
IDlabel_dict = {v: k for k, v in labelID_dict.items()}
word_index = pickle.load(open("./data/Hooklog/MikeGithub/single_dict_enc_Mike.pkl","rb"))
word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
word_idx_pd['ori_word'] = word_idx_pd.index 
word_idx_pd = word_idx_pd.set_index(["ID"])
print(word_idx_pd)

pred_label = list(np.argmax(pred_label,axis=-1))
def getKeysByValues(listOfValues,dfOfElements=word_idx_pd):
    ori_word_li = []
    for item in listOfValues:
        key = item
#         print(key)
        if key == 0:
            continue
        ori_word = dfOfElements.loc[key][0]
        ori_word_li.append(ori_word)
    return ori_word_li

                 ori_word
ID                       
1          WinHttpConnect
2                 WinExec
3             OpenProcess
4           RegQueryValue
5                CopyFile
6            InternetOpen
7      WinHttpOpenRequest
8      WinHttpSendRequest
9            RegCreateKey
10           RegDeleteKey
11            LoadLibrary
12        InternetConnect
13        HttpSendRequest
14           CreateThread
15             DeleteFile
16            ExitProcess
17            WinHttpOpen
18            RegSetValue
19             CreateFile
20          CreateProcess
21           RegEnumValue
22  CreateProcessInternal
23     CreateRemoteThread


In [84]:
word_fil_text = []
word_ori_text = []
from tqdm import tqdm
for text in tqdm(int_ori_text): #text = one epoch
    temp = []
    for t in text: #t = one sample
        temp.append(getKeysByValues(list(t)))
    word_ori_text.append(temp)
for text in tqdm(int_fil_text):
    temp = []
    for t in text:
        temp.append(getKeysByValues(list(t)))
    word_fil_text.append(temp)

save_path = save_dir + "original_words.pkl"
pickle.dump(file=open(save_path,'wb'),obj=word_ori_text)
save_path = save_dir + "filterUsed_words.pkl"
pickle.dump(file=open(save_path,'wb'),obj=word_fil_text)
print(save_path)

100%|██████████| 2000/2000 [03:42<00:00,  8.98it/s]


./results/Hooklog/2020012101/filterUsed_words.pkl


In [85]:
col = ['Epoch#','text_id','ori_text','filter_text','label','predict']
df = pd.DataFrame(columns=col)
for epoch,(text_ori,text_fil,labelID,predID) in tqdm(enumerate(zip(word_ori_text,word_fil_text,int_label,pred_label))):
    for idx,(t_o,t_f,label,pred) in enumerate(zip(text_ori,text_fil,labelID,predID)):
        temp = pd.Series([epoch+1,idx," ".join([str(x) for x in t_o])," ".join([str(x) for x in t_f]),IDlabel_dict[label],IDlabel_dict[pred]],col)
        df = df.append(temp,ignore_index=True)

save_path = save_dir + "caseStudy_result.xlsx"
df.to_excel(save_path,index=False)
print(save_path)
df

2000it [09:33,  3.49it/s]


./results/Hooklog/2020012101/caseStudy_result.xlsx


,epoch#,text_id,ori_text,filter_text,label,predict
0,0,0,LoadLibrary LoadLibrary LoadLibrary RegQueryVa...,LoadLibrary LoadLibrary LoadLibrary RegQueryVa...,lydra_woj_g_44,firseria_woj_g_144+
1,0,1,LoadLibrary LoadLibrary LoadLibrary LoadLibrar...,LoadLibrary LoadLibrary LoadLibrary LoadLibrar...,allaple_woj_g_98,sytro_woj_g_166
2,0,2,LoadLibrary LoadLibrary LoadLibrary CreateFile...,LoadLibrary LoadLibrary LoadLibrary CreateFile...,madang_woj_g_137,sytro_woj_g_166
3,0,3,LoadLibrary LoadLibrary LoadLibrary LoadLibrar...,LoadLibrary LoadLibrary LoadLibrary LoadLibrar...,loadmoney2_woj_g_189,morstar_old
4,0,4,LoadLibrary LoadLibrary LoadLibrary RegQueryVa...,LoadLibrary LoadLibrary LoadLibrary RegQueryVa...,elkern_woj_g_127,firseria_woj_g_144+
...,...,...,...,...,...,...
97995,1999,44,LoadLibrary LoadLibrary LoadLibrary RegQueryVa...,,firseria_woj_g_144+,bettersurf_woj_g_137+
97996,1999,45,LoadLibrary LoadLibrary LoadLibrary RegQueryVa...,LoadLibrary LoadLibrary LoadLibrary RegQueryVa...,sytro_woj_g_166,sytro_woj_g_166
97997,1999,46,LoadLibrary LoadLibrary LoadLibrary RegQueryVa...,,bettersurf_woj_g_137+,bettersurf_woj_g_137+
97998,1999,47,LoadLibrary LoadLibrary LoadLibrary RegQueryVa...,,firseria_woj_g_144+,bettersurf_woj_g_137+


todo: 
* 填hyperparameters
* cosine similarity比較